In [1]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt
from decouple import config
import json


In [2]:
key_path = os.environ.get("KEY_PATH")
# service_account_info = json.loads(key_path)


In [3]:
# [START bigquery_client_json_credentials]

# TODO(developer): Set key_path to the path to the service account key
#                  file.
# key_path = "path/to/service_account.json"

# credentials = service_account.Credentials.from_service_account_file(
#     key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
# )

# credentials = service_account.Credentials.from_service_account_info(service_account_info)

# credentials = service_account.Credentials.from_service_account(service_account_email = os.environ.get("client_email"), signer = os.getenv("private_key_id"))

credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
# [END bigquery_client_json_credentials]


In [4]:
# query = '''SELECT * FROM `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
# WHERE Timestamp BETWEEN DATETIME("2023-08-17")
# AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY) and Leg_0_StopsCount is not null'''

query = '''Select * from `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
WHERE Timestamp BETWEEN DATETIME("2023-08-17")AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY)
AND CabinClass = 'ECONOMY'
ORDER BY RAND()
LIMIT 200
'''


In [5]:
query_job = client.query(query)


In [6]:

results = query_job.result()

the_data = results.to_dataframe()


In [7]:
the_data


,Guid,Timestamp,Market,Locale,Currency,OriginAirport,OriginCity,OriginCountry,DestinationAirport,DestinationCity,...,Leg_1_segment_3_OriginName,Leg_1_segment_3_DestinationIATA,Leg_1_segment_3_DestinationName,Leg_1_segment_3_DepartureTimestamp,Leg_1_segment_3_ArrivalTimestamp,Leg_1_segment_3_MarketingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_1_segment_3_FlightNumber,Leg_1_segment_3_DurationMin,Leg_1_segment_3_TravelDistanceKm
0,5411238c-3ce6-11ee-bde8-bef9c112ef24,2023-08-17 10:10:44,UK,en-GB,GBP,DEL,DEL,IN,GOI,GOI,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
1,8f08c722-3cf7-11ee-a9d6-bef9c112ef24,2023-08-17 12:14:04,US,en-US,USD,YUL,YMQ,CA,YWG,YWG,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
2,35a81760-3d87-11ee-af9c-4605cb41ae37,2023-08-18 05:22:22,BR,pt-BR,BRL,MIA,MIA,US,LGW,LON,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
3,39e64578-3ce6-11ee-b301-fe5d4e0868a8,2023-08-17 10:10:00,KR,ko-KR,KRW,PUS,PUS,KR,CTS,SPK,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
4,cb2e8ba4-3dd0-11ee-8311-eeb4e704f759,2023-08-18 14:09:06,DE,de-DE,EUR,ZRH,ZRH,CH,PMI,PMI,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,90d74698-3d8b-11ee-b2ad-365025c0897b,2023-08-18 05:53:33,RU,ru-RU,RUB,EVN,EVN,AM,VIE,VIE,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
196,b1361546-3d1c-11ee-818b-b2667385bc36,2023-08-17 16:39:53,IN,en-GB,INR,LKO,LKO,IN,AMD,AMD,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
197,fdf5140e-3d3f-11ee-a466-7e30eda1f70e,2023-08-17 20:52:34,UK,es-ES,COP,MDE,MDE,CO,MEX,MEX,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>
198,2c8e4ef8-3d35-11ee-a60e-36022e6df2ef,2023-08-17 19:35:08,RS,en-GB,RSD,BEG,BEG,RS,NAP,NAP,...,,,,NaT,NaT,,,<NA>,<NA>,<NA>


In [18]:
the_data['ItinerarySortType'].value_counts(normalize=True)


BEST                      0.536910
                          0.357785
CHEAPEST                  0.094840
FASTEST                   0.008325
DEPARTURE_OUTBOUND_ASC    0.002090
DEPARTURE_INBOUND_ASC     0.000050
Name: ItinerarySortType, dtype: float64

In [19]:
the_data['CabinClass'].value_counts(normalize=True)


ECONOMY    1.0
Name: CabinClass, dtype: float64

In [21]:
colums_subset = ['Market', 'OriginAirport', 'DestinationAirport', 'Locale', 'CabinClass', 'Kind',
          'TripLengthNights', 'TravelType', 'TravelDistanceKm', 'TravelHorizonDays', 'RedirectType', 'ItineraryPosition', 'ItinerarySortType',
          'PriceCustomCurrencyPerPax',
          'Leg_0_StopsCount', 'Leg_1_StopsCount',
          'Leg_0_segment_0_OperatingCarrierIATA', 'Leg_0_segment_1_OperatingCarrierIATA', 'Leg_0_segment_2_OperatingCarrierIATA', 'Leg_0_segment_3_OperatingCarrierIATA',
          'Leg_1_segment_0_OperatingCarrierIATA', 'Leg_1_segment_1_OperatingCarrierIATA', 'Leg_1_segment_2_OperatingCarrierIATA', 'Leg_1_segment_3_OperatingCarrierIATA',
          'Leg_0_DurationMin', 'Leg_1_DurationMin'
]

all_columns = list(the_data.columns)


In [22]:
remaining_columns = list(set(all_columns) - set(colums_subset))
remaining_columns


['UserDistrictName',
 'Leg_1_segment_3_DestinationName',
 'Device',
 'Leg_1_segment_2_DepartureTimestamp',
 'Leg_1_segment_0_DestinationIATA',
 'PriceUserCurrencyPerPax',
 'ThemeFoodWeight',
 'OSName',
 'Leg_1_segment_1_TravelDistanceKm',
 'OSVersion',
 'SelfTransfer',
 'Leg_0_segment_2_OriginName',
 'Leg_0_segment_2_MarketingCarrierIATA',
 'Leg_1_segment_3_DepartureTimestamp',
 'Currency',
 'Leg_1_segment_0_OriginName',
 'Leg_0_segment_0_MarketingCarrierIATA',
 'UserRegionName',
 'DestinationCountry',
 'SearchLowestPriceRatio',
 'PriceUserCurrency',
 'Leg_0_segment_3_DepartureTimestamp',
 'ThemeBeachWeight',
 'Leg_0_segment_3_MarketingCarrierIATA',
 'ProviderType',
 'Leg_1_segment_2_DestinationIATA',
 'Leg_0_segment_3_OriginIATA',
 'Leg_1_segment_0_TravelDistanceKm',
 'ProviderPreference',
 'Leg_1_segment_0_FlightNumber',
 'Leg_0_segment_0_DestinationIATA',
 'PriceCustomCurrency',
 'Leg_0_segment_2_DurationMin',
 'Leg_1_segment_2_MarketingCarrierIATA',
 'Leg_1_segment_0_ArrivalTimesta

In [23]:
data_subset = the_data[colums_subset].copy()
data_subset


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_0_OperatingCarrierIATA,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin
0,BR,GYN,SLZ,pt-BR,ECONOMY,ONE_WAY,NaN,DOMESTIC,1655.0,139,...,G3,G3,G3,,,,,,480,NaN
1,UK,SAW,MAN,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,2729.0,11,...,PC,,,,,,,,260,NaN
2,UK,MDE,CTG,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,476.0,0,...,AV,,,,,,,,73,NaN
3,IT,MXP,SSH,it-IT,ECONOMY,RETURN,9.0,INTERNATIONAL,2989.0,70,...,U2,,,,W4,,,,195,290.0
4,AR,EZE,FCO,es-MX,ECONOMY,RETURN,78.0,INTERNATIONAL,11155.0,67,...,AZ,,,,AZ,,,,775,850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,IT,FOG,LIN,it-IT,ECONOMY,ONE_WAY,NaN,DOMESTIC,674.0,51,...,L9,,,,,,,,90,NaN
199996,DE,CAI,JFK,de-DE,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,9017.0,16,...,SV,SV,,,,,,,1085,NaN
199997,EG,HRG,SAW,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1580.0,40,...,PC,,,,,,,,160,NaN
199998,DE,HAM,BUD,de-DE,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,948.0,3,...,EW,,,,,,,,100,NaN


In [24]:
data_subset['OD'] = data_subset['OriginAirport'] + data_subset['DestinationAirport']


In [25]:
data_subset[data_subset['CabinClass'] == 'ECONOMY']


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin,OD
0,BR,GYN,SLZ,pt-BR,ECONOMY,ONE_WAY,NaN,DOMESTIC,1655.0,139,...,G3,G3,,,,,,480,NaN,GYNSLZ
1,UK,SAW,MAN,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,2729.0,11,...,,,,,,,,260,NaN,SAWMAN
2,UK,MDE,CTG,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,476.0,0,...,,,,,,,,73,NaN,MDECTG
3,IT,MXP,SSH,it-IT,ECONOMY,RETURN,9.0,INTERNATIONAL,2989.0,70,...,,,,W4,,,,195,290.0,MXPSSH
4,AR,EZE,FCO,es-MX,ECONOMY,RETURN,78.0,INTERNATIONAL,11155.0,67,...,,,,AZ,,,,775,850.0,EZEFCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,IT,FOG,LIN,it-IT,ECONOMY,ONE_WAY,NaN,DOMESTIC,674.0,51,...,,,,,,,,90,NaN,FOGLIN
199996,DE,CAI,JFK,de-DE,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,9017.0,16,...,SV,,,,,,,1085,NaN,CAIJFK
199997,EG,HRG,SAW,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1580.0,40,...,,,,,,,,160,NaN,HRGSAW
199998,DE,HAM,BUD,de-DE,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,948.0,3,...,,,,,,,,100,NaN,HAMBUD


In [28]:
data_subset['Leg_0_segment_0_OperatingCarrierIATA'].value_counts(normalize=True)[:30]


FR    0.047975
      0.043635
6E    0.037620
U2    0.035985
UK    0.022780
VY    0.021410
TK    0.019665
EK    0.019245
W4    0.018285
PC    0.017340
G3    0.017160
LH    0.016450
W6    0.016420
AI    0.016105
BA    0.015205
QR    0.015000
JJ    0.012855
SG    0.010360
EY    0.010235
OZ    0.010070
7C    0.009780
SV    0.009640
FD    0.009380
TW    0.009170
QP    0.009080
LJ    0.009015
CX    0.009000
AF    0.008960
RR    0.008845
VJ    0.008710
Name: Leg_0_segment_0_OperatingCarrierIATA, dtype: float64